In [1]:
#Please enter here the paths of the training file and the test file
training_file = "Untitled Folder/Datasets/seq2tree_atis/train.txt"
test_file = "Untitled Folder/Datasets/seq2tree_atis/test.txt"

"Untitled Folder/Datasets/seq2tree_atis/test.txt"

In [1]:
#Please enter here the paths of the training file and the test file
training_file = "Untitled Folder/Datasets/seq2tree_geoqueries/train.txt"
test_file = "Untitled Folder/Datasets/seq2tree_geoqueries/test.txt"

"Untitled Folder/Datasets/seq2tree_geoqueries/test.txt"

In [1]:
training_file = "Untitled Folder/Datasets/seq2tree_jobqueries/train.txt"
test_file = "Untitled Folder/Datasets/seq2tree_jobqueries/test.txt"

"Untitled Folder/Datasets/seq2tree_jobqueries/test.txt"

In [2]:
using Knet: Knet, AutoGrad, param, param0, mat, RNN, relu, Data, adam, progress, nll, zeroone
using Distributions
import .Iterators: cycle, Cycle, take
using IterTools
import CUDA
using CUDA

In [3]:
function tok_int(training_file)
    
    #global int2tok_input, int2tok_output
    f = open(training_file)
    tok2int_input = Dict{String,Int}() #keys: unique input tokens. values: id number of the token.
    int2tok_input = Vector{String}() #indices: numbers. values: tokens corresponding to those numbers.
    push!(int2tok_input, "<s>") #start token
    push!(int2tok_input, "</s>") #stop token
    tok2int_input["<s>"] = 1
    tok2int_input["</s>"] = 2
    tok2int_output = Dict{String,Int}() #keys: unique output tokens. values: id number of the token.
    int2tok_output = Vector{String}() #keys: numbers. values: tokens corresponding to those numbers.
    push!(int2tok_output, "<s>") #start token
    push!(int2tok_output, "</s>") #stop token
    tok2int_output["<s>"] = 1
    tok2int_output["</s>"] = 2
    while ! eof(f)
        seq = readline(f)
        seq = chomp(seq)
        input, output = split(seq, "\t")
        tokens = split(input, " ")
        for token in tokens
            if !haskey(tok2int_input, token)
                push!(int2tok_input, token)
                tok2int_input[token] = length(int2tok_input)
            end            
        end
        tokens = split(output, " ")
        for token in tokens
            if !haskey(tok2int_output, token)
                push!(int2tok_output, token)
                tok2int_output[token] = length(int2tok_output)
            end           
        end        
    end
    push!(int2tok_input, "UNK") 
    tok2int_input["UNK"] = 123
    push!(int2tok_output, "UNK") 
    tok2int_output["UNK"] = 123
    Vq = length(int2tok_input) #number of unique input tokens
    Va = length(int2tok_output) #number of unique output tokens    
    return int2tok_input, tok2int_input, int2tok_output, tok2int_output, Va, Vq
end

tok_int (generic function with 1 method)

In [4]:
function data_reader(training_file, tok2int_input, tok2int_output)
    
    data = []
    f = open(training_file)
    while ! eof(f)
        seq = readline(f)
        seq = chomp(seq)
        input, output = split(seq, "\t")
        tokens = split(input, " ")
        s = Vector{Int}() #vector that stores the token ids.
        global n_in = 0 #just to check the number of unknown tokens
        for token in tokens            
            if haskey(tok2int_input, token)
                push!(s, tok2int_input[token])
            else
                push!(s, tok2int_input["UNK"])
                n_in += 1
            end 
        end        

        tokens = split(output, " ")
        s2 = Vector{Int}() #vector that stores the token ids.
        global n_out = 0 #just to check the number of unknown tokens
        for token in tokens
            if haskey(tok2int_output, token)
                push!(s2, tok2int_output[token])
            else
                push!(s2, tok2int_output["UNK"])
                n_out += 1
            end
        end 
        push!(data, (s, s2))
    end    
    return data
end

data_reader (generic function with 1 method)

In [5]:
function minibatch(data,batchsize)
    n = length(data)
    batch_data = Any[]
    p = 0
    while p + batchsize <= n
        max_seq_len = length(data[p+batchsize][1])
        enc_seq = zeros(Int64, batchsize, max_seq_len+2) 
        enc_seq[:,1] .= 1
        for i in 1:batchsize
            seq = data[p+i][1]
            seq_len = length(seq)
            for j in 1:seq_len
                enc_seq[i,j+1] = seq[seq_len-j+1]
            end
            for k in seq_len+2:max_seq_len+2 
                enc_seq[i,k] = 2
            end
        end
        max_seq_len = -1
        for i in 1:batchsize
            seq = data[p+i][2]
            if length(seq) > max_seq_len 
                max_seq_len = length(seq)
            end
        end
        dec_seq = zeros(Int64, batchsize, max_seq_len+2)
        dec_seq[:,1] .= 1
        for i in 1:batchsize
            seq = data[p+i][2]
            seq_len = length(seq)
            for j in 1:seq_len
                dec_seq[i,j + 1] = seq[j]
            end
            dec_seq[i,(seq_len + 2):end] .= 2
        end
        p += batchsize
        push!(batch_data, (enc_seq, dec_seq))
    end
    return batch_data
end

minibatch (generic function with 1 method)

In [6]:
function mask(a, pad)
    a = copy(a)
    for i in 1:size(a, 1)
        j = size(a,2)
        while a[i, j] == pad && j > 1
            if a[i, j - 1] == pad
                a[i, j] = 0
            end
            j -= 1
        end
    end
    return a
end

mask (generic function with 1 method)

In [7]:
struct Embed
    w
end
function Embed(embedsize::Int, vocabsize::Int)
    w = rand(Distributions.Uniform(-0.08,0.08),embedsize,vocabsize)
    w = Knet.Param(convert(Knet.KnetArray{Float32},w))
    return Embed(w)
end
(e::Embed)(x) = e.w[:,x] #x: word id

struct Linear
    w
    b
    f
end
function Linear(i::Int,o::Int,f=identity)
    w = rand(Distributions.Uniform(-0.08,0.08),o,i)
    w = Knet.Param(convert(Knet.KnetArray{Float32},w))
    b = zeros(o)
    b = Knet.Param(convert(Knet.KnetArray{Float32},b))
    return Linear(w,b,f)
end
#Linear(i::Int,o::Int,f=identity) = Linear(param(o,i), param0(o), f)
(d::Linear)(x) = d.f.(d.w * mat(x,dims=1) .+ d.b)

In [8]:
mutable struct Tree
    parent
    num_children
    children
end

In [9]:
struct seq2seq
    input_embed
    output_embed
    encoder
    decoder
    linear
    linear_att
    dropout
    
end

function seq2seq(X::Int, H::Int, Vq::Int, Va::Int, dropout::Real)
    a = Embed(X, Vq)
    b = Embed(X, Va)
    c = RNN(X, H; rnnType=:lstm, numLayers=1, dropout=dropout)
    d = RNN(X, H; rnnType=:lstm, numLayers=1, dropout=dropout)
    e = Linear(H,Va)
    f = Linear(2*H, H, tanh)
    g = dropout
    return seq2seq(a, b, c, d, e, f, g)
end



seq2seq

In [10]:
function get_attention_vectors(s::seq2seq, encoder_out, decoder_out, mode::Int)
    
    T = size(decoder_out,3)
    h_enc = permutedims(encoder_out, [3, 1, 2]) # T,H,B
    local attention
   
    h_dec = permutedims(decoder_out[:,:,1:1], [1, 3, 2]) # H,1,B
    dot = Knet.bmm(h_enc, h_dec) # T,1,B
    dot = permutedims(dot, [3, 1, 2]) # B,T,1
    dot = reshape(dot, (size(dot,1), size(dot,2))) # B,T
    attn = Knet.softmax(dot, dims=2) # B,T
    attn = reshape(attn, (size(attn,1), size(attn,2), 1)) # B,T,1
    attn = permutedims(attn, [2, 3, 1]) # T,1,B
    h_enc_tr = permutedims(encoder_out, [1, 3, 2]) # H,T,B
    context = Knet.bmm(h_enc_tr, attn) # H,1,B
    cat_out = cat(h_dec,context; dims=1) # 2H,1,B
    cat_out = permutedims(cat_out, [1, 3, 2]) # 2H,B,1
    cat_out = reshape(cat_out, (size(cat_out,1), size(cat_out,2))) # 2H,B
    h_att = s.linear_att(cat_out) # H,B    
    attention = h_att
    if mode == 2
        attention = reshape(attention, (size(attention,1), size(attention,2), 1)) # H,B,1
        return attention # H,B,1
    elseif mode == 1
        for i in 2:T
            h_dec = permutedims(decoder_out[:,:,i:i], [1, 3, 2]) # H,1,B
            dot = Knet.bmm(h_enc, h_dec) # T,1,B
            dot = permutedims(dot, [3, 1, 2]) # B,T,1
            dot = reshape(dot, (size(dot,1), size(dot,2))) # B,T
            attn = Knet.softmax(dot, dims=2) # B,T
            attn = reshape(attn, (size(attn,1), size(attn,2), 1)) # B,T,1
            attn = permutedims(attn, [2, 3, 1]) # T,1,B
            h_enc_tr = permutedims(encoder_out, [1, 3, 2]) # H,T,B
            context = Knet.bmm(h_enc_tr, attn) # H,1,B
            cat_out = cat(h_dec,context; dims=1) # 2H,1,B
            cat_out = permutedims(cat_out, [1, 3, 2]) # 2H,B,1
            cat_out = reshape(cat_out, (size(cat_out,1), size(cat_out,2))) # 2H,B
            h_att = s.linear_att(cat_out) # H,B        
            attention = cat(attention, h_att; dims=3)            
        end
        return attention # H,B,T
    end
end

get_attention_vectors (generic function with 1 method)

In [11]:
function (s::seq2seq)(x, y; average=true) 
    # x: B,Tmaxx y: B,Tmaxy
    s.encoder.h, s.encoder.c = 0, 0
    x_embedded = s.input_embed(x) # X,B,Tmaxx
    y_embedded = s.output_embed(y) # X,B,Tmaxy
    encoder_out = s.encoder(x_embedded) # H,B,Tmaxx
    s.decoder.h, s.decoder.c = s.encoder.h, s.encoder.c    
    decoder_out = s.decoder(y_embedded[:, :, 1:end-1]) # H,B,Tmaxy-1
    attention = get_attention_vectors(s, encoder_out, decoder_out, 1)
    dims = size(attention) # H,B,Tmaxx-1
    output = s.linear(Knet.dropout(reshape(attention, dims[1], dims[2] * dims[3]), s.dropout))
    scores = reshape(output, size(output, 1), dims[2], dims[3])
    nll(scores, mask(y[:, 2:end], 2); dims=1, average=average)
end

In [12]:
### FOR PREDICTION/TESTING
function (s::seq2seq)(x; stopfactor = 4)
    # x: B, Tmax
    s.encoder.h, s.encoder.c = 0, 0
    x_embedded = s.input_embed(x)
    encoder_out = s.encoder(x_embedded)
    s.decoder.h, s.decoder.c = s.encoder.h, s.encoder.c
    step = fill(1, size(x)[1], 1) # B,1 --full of 1's: start tokens
    out = fill(2, size(x)[1], 0) # B,0
    stopping_criteria = fill(false, size(x)[1])
    for i in 1:(stopfactor * size(x,2))
        if sum(stopping_criteria) == size(x)[1]
            break
        end
        stepembed = s.output_embed(step) # X,B,1
        decoder_out = s.decoder(stepembed) # H,B,1
        attention = get_attention_vectors(s, encoder_out, decoder_out, 2) # H,B,1
        dims = size(attention) # H,B,1
        output = s.linear(reshape(attention, dims[1], dims[2] * dims[3])) # H,B
        step = transpose([i[1] for i in argmax(output; dims=1)]) # B,1
        out = hcat(out, step) # B,?
        for e in findall(x -> x == 2, step)
            stopping_criteria[e[1]] = true
        end
    end
    return out
end

In [13]:
function model_accuracy(model, data)
    total = 0
    no_sequences = 0
    for (x, y) in data
        y_pred = model(x)
        for i in 1:size(x, 1)
            no_sequences += 1            
            y_clipped = []
            y_pred_clipped = []
            for j in 1:size(y,2)                
                if y[i,j] == 2                   
                    append!(y_clipped, y[i,2:j])
                    break
                end
            end            
            for j in 1:size(y_pred,2)
                if y_pred[i,j] == 2 
                    append!(y_pred_clipped, y_pred[i,1:j])
                    break
                end
            end
            if length(y_clipped) == length(y_pred_clipped)
                if sum(y_pred_clipped .== y_clipped) == size(y_clipped,1)                   
                    total += 1                    
                end
            end
        end
    end
    return total / no_sequences    
end

model_accuracy (generic function with 1 method)

In [14]:
lrdecay!(s::seq2seq, decay::Real) =
    for p in Knet.params(s); p.opt.lr = p.opt.lr*decay; end

lrdecay! (generic function with 1 method)

In [15]:
int2tok_input, tok2int_input, int2tok_output, tok2int_output, Va, Vq = tok_int(training_file)
trndata = data_reader(training_file, tok2int_input, tok2int_output)
tstdata = data_reader(test_file, tok2int_input, tok2int_output)
X = 200
H = 200 
trndata_batch = minibatch(trndata, 20)
tstdata_batch = minibatch(tstdata, 20)

7-element Array{Any,1}:
 ([1 29 … 2 2; 1 14 … 2 2; … ; 1 20 … 34 2; 1 18 … 34 2], [1 3 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3 … 2 2])
 ([1 16 … 2 2; 1 20 … 2 2; … ; 1 16 … 25 2; 1 20 … 25 2], [1 3 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3 … 2 2])
 ([1 62 … 2 2; 1 14 … 2 2; … ; 1 18 … 5 2; 1 12 … 79 2], [1 3 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3 … 2 2])
 ([1 18 … 2 2; 1 52 … 2 2; … ; 1 16 … 59 2; 1 26 … 5 2], [1 19 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3 … 2 2])
 ([1 18 … 2 2; 1 99 … 2 2; … ; 1 14 … 9 2; 1 24 … 5 2], [1 3 … 2 2; 1 3 … 2 2; … ; 1 22 … 6 2; 1 3 … 2 2])
 ([1 14 … 2 2; 1 45 … 2 2; … ; 1 86 … 5 2; 1 14 … 5 2], [1 3 … 2 2; 1 3 … 2 2; … ; 1 3 … 2 2; 1 3 … 2 2])
 ([1 18 … 2 2; 1 31 … 2 2; … ; 1 26 … 2 2; 1 18 … 9 2], [1 3 … 2 2; 1 8 … 2 2; … ; 1 3 … 2 2; 1 3 … 6 2])

In [19]:
my_model = seq2seq(X,H,Vq,Va,0.4) # dropout=0.4 for GEO and JOBS datasets, dropout = 0.3 for ATIS dataset.

seq2seq(Embed(P(Knet.KnetArrays.KnetArray{Float32,2}(200,173))), Embed(P(Knet.KnetArrays.KnetArray{Float32,2}(200,50))), LSTM(input=200,hidden=200,dropout=0.4), LSTM(input=200,hidden=200,dropout=0.4), Linear(P(Knet.KnetArrays.KnetArray{Float32,2}(50,200)), P(Knet.KnetArrays.KnetArray{Float32,1}(50)), identity), Linear(P(Knet.KnetArrays.KnetArray{Float32,2}(200,400)), P(Knet.KnetArrays.KnetArray{Float32,1}(200)), tanh), 0.4)

In [20]:
for i in 1:200
    Knet.rmsprop!(my_model, trndata_batch;rho=0.95,gclip=5)
    if i%5 == 0
        println("After $i epochs")
        #println("train accuracy: ", model_accuracy(my_model, trndata_batch))
        println("test accuracy: ", model_accuracy(my_model, tstdata_batch))
    end
end

After 5 epochs
test accuracy: 0.0
After 10 epochs
test accuracy: 0.1
After 15 epochs
test accuracy: 0.0
After 20 epochs
test accuracy: 0.12857142857142856
After 25 epochs
test accuracy: 0.0
After 30 epochs
test accuracy: 0.40714285714285714
After 35 epochs
test accuracy: 0.6285714285714286
After 40 epochs
test accuracy: 0.6857142857142857
After 45 epochs
test accuracy: 0.6857142857142857
After 50 epochs
test accuracy: 0.7571428571428571
After 55 epochs
test accuracy: 0.7714285714285715
After 60 epochs
test accuracy: 0.7571428571428571
After 65 epochs
test accuracy: 0.7785714285714286
After 70 epochs
test accuracy: 0.7785714285714286
After 75 epochs
test accuracy: 0.6857142857142857
After 80 epochs
test accuracy: 0.7928571428571428
After 85 epochs
test accuracy: 0.7714285714285715
After 90 epochs
test accuracy: 0.7928571428571428
After 95 epochs
test accuracy: 0.8071428571428572
After 100 epochs
test accuracy: 0.7857142857142857
After 105 epochs
test accuracy: 0.7785714285714286
After 1